In [ ]:
# import os
# import glob
# import cv2
# import supervision as sv
# from rfdetr import RFDETRNano

# # 1. Initialize the Model with the EXACT settings found in the nano checkpoint
# model = RFDETRNano(
#     patch_size=16,                # Matches the [16, 16] weight shape
#     positional_encoding_size=24,   # Matches the 577 embedding shape
#     resolution=384,               # Matches the default Nano training res
#     out_feature_indexes=[3, 6, 9, 12],
#     num_windows=2,
#     dec_layers=2,
#     pretrain_weights="rf-detr-nano.pth"
# )

# # 2. Setup Visualization
# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# # 3. Paths
# TEST_DIR = r"D:\Projects\RF_DETR_Wetland\dataset\test"
# OUTPUT_DIR = "test_results_fixed_shapes"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# # 4. Inference Loop
# image_paths = glob.glob(os.path.join(TEST_DIR, "*.jpg"))[:15]

# print("Model loaded successfully! Running inference...")

# # for img_path in image_paths:
# #     image = cv2.imread(img_path)
# #     if image is None: continue
    
# #     # Run prediction with a VERY low confidence to see raw "guesses"
# #     results = model.predict(image, confidence=0.01)
    
# #     # LOGGING: Print every class ID the model found
# #     found_classes = set(results.class_id)
# #     print(f"Image {os.path.basename(img_path)} raw detections: {found_classes}")
    
# #     # If the model is seeing SOMETHING, let's visualize everything it sees
# #     if len(results.class_id) > 0:
# #         labels = [f"ID:{c} {conf:.2f}" for c, conf in zip(results.class_id, results.confidence)]
# #         annotated_img = box_annotator.annotate(scene=image.copy(), detections=results)
# #         annotated_img = label_annotator.annotate(scene=annotated_img, detections=results, labels=labels)
# #         cv2.imwrite(os.path.join(OUTPUT_DIR, "diag_" + os.path.basename(img_path)), annotated_img)
# #         print(f"Processed {os.path.basename(img_path)} - {len(results)} detections found.")
        
# # print(f"\nDone! Results in {OUTPUT_DIR}")

# for img_path in image_paths:
#     image = cv2.imread(img_path)
#     if image is None: continue
    
#     # Run prediction - using a lower confidence to ensure we see detections
#     results = model.predict(image, confidence=0.10)
    
#     # Class 15 is Bird in COCO
#     bird_detections = results[results.class_id == 16]
    
#     # Visualization
#     labels = [f"Bird {conf:.2f}" for conf in bird_detections.confidence]
#     annotated_img = box_annotator.annotate(scene=image.copy(), detections=bird_detections)
#     annotated_img = label_annotator.annotate(scene=annotated_img, detections=bird_detections, labels=labels)
    
#     cv2.imwrite(os.path.join(OUTPUT_DIR, os.path.basename(img_path)), annotated_img)
#     print(f"Processed {os.path.basename(img_path)} - {len(bird_detections)} birds found.")

# print(f"\nDone! Results in {OUTPUT_DIR}")



In [ ]:
import os
import glob
import cv2
import supervision as sv
from rfdetr import RFDETRNano

# 1. Initialize Model
model = RFDETRNano(
    patch_size=16,
    positional_encoding_size=24,
    resolution=384,
    out_feature_indexes=[3, 6, 9, 12],
    num_windows=2,
    dec_layers=2,
    pretrain_weights="rf-detr-nano.pth"
)

# 2. Get the confirmed Class Names from the model
# We know from your diagnostic that ID 16 = bird
CLASS_NAMES = model.class_names

# 3. Setup Visualization
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# 4. Paths
TEST_DIR = r"D:\Projects\RF_DETR_Wetland\dataset\test"
OUTPUT_DIR = r"D:\Projects\RF_DETR_Wetland\test_results_final"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 5. Inference Loop
image_paths = glob.glob(os.path.join(TEST_DIR, "*.jpg"))[:20]
print(f"Verified: ID 16 is '{CLASS_NAMES[16]}'. Starting inference...")

for img_path in image_paths:
    image = cv2.imread(img_path)
    if image is None: continue
    
    # Run prediction - we only care about Bird (ID 16)
    results = model.predict(image, confidence=0.15)
    
    # Filter for ID 16
    bird_detections = results[results.class_id == 16]
    
    if len(bird_detections) > 0:
        # Generate labels using the model's confirmed name for ID 16
        labels = [f"{CLASS_NAMES[16]} {conf:.2f}" for conf in bird_detections.confidence]
        
        annotated_img = box_annotator.annotate(scene=image.copy(), detections=bird_detections)
        annotated_img = label_annotator.annotate(scene=annotated_img, detections=bird_detections, labels=labels)
        
        cv2.imwrite(os.path.join(OUTPUT_DIR, os.path.basename(img_path)), annotated_img)
        print(f"Processed {os.path.basename(img_path)}: Found {len(bird_detections)} birds.")
    else:
        print(f"Processed {os.path.basename(img_path)}: No birds detected.")

print(f"\nSuccess! Check folder : {OUTPUT_DIR}")

In [ ]:
import torch
from rfdetr import RFDETRNano
from rfdetr.util.coco_classes import COCO_CLASSES

# 1. Initialize the model
model = RFDETRNano(
    patch_size=16,
    positional_encoding_size=24,
    resolution=384,
    pretrain_weights="rf-detr-nano.pth"
)

print("-" * 30)
print("CLASS ID DIAGNOSTIC")
print("-" * 30)

# Check A: Internal model attribute (Common in Roboflow models)
if hasattr(model, 'class_names'):
    print("Found 'class_names' on model object:")
    names = model.class_names # Usually a dict {id: name}
    for i in range(len(names)):
        print(f"ID {i}: {names.get(i, '??')}")

# Check B: Utility import (The standard for RF-DETR)
elif 'COCO_CLASSES' in globals() or 'COCO_CLASSES' in locals():
    print("Found COCO_CLASSES in utility library:")
    # Some COCO lists start at 0 (Bird=14), others at 1 (Bird=15)
    for i, name in enumerate(COCO_CLASSES):
        print(f"Index {i}: {name}")

# Check C: Raw Inspection of the classification head
else:
    num_classes = model.model.class_embed[0].out_features
    print(f"Model head detected with {num_classes} output slots.")
    print("No string names found in metadata. Using standard COCO mapping.")

print("-" * 30)

In [16]:
import os
import glob
import cv2
import pandas as pd
import random  # Added for random sampling
import supervision as sv
from rfdetr import RFDETRNano

# 1. Initialize Model
model = RFDETRNano(
    patch_size=16,
    positional_encoding_size=24,
    resolution=384,
    out_feature_indexes=[3, 6, 9, 12],
    num_windows=2,
    dec_layers=2,
    pretrain_weights="rf-detr-nano.pth"
)

CLASS_NAMES = model.class_names
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# 2. Paths
TEST_DIR = r"D:\Projects\RF_DETR_Wetland\dataset\test"
OUTPUT_DIR = r"D:\Projects\RF_DETR_Wetland\test_results_final"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 3. Data Collection Setup
detection_summary = []

# --- NEW STEP: RANDOM SELECTION ---
# Get ALL images first
image_paths = glob.glob(os.path.join(TEST_DIR, "*.jpg"))[:500]  # Limit to 500 for large datasets
total_images = len(image_paths)

# Pick 50 random image names to save (ensure we don't try to pick more than exist)
num_to_save = min(50, total_images)
images_to_save = set(random.sample(image_paths, num_to_save))
print(f"Total images to process: {total_images}")
print(f"Random images to be saved: {num_to_save}")
# ---------------------------------

print(f"Verified: ID 16 is '{CLASS_NAMES[16]}'. Starting inference...")

for img_path in image_paths:
    img_name = os.path.basename(img_path)
    image = cv2.imread(img_path)
    if image is None: continue
    
    # 1. Always run detection for the summary reports
    results = model.predict(image, confidence=0.15)
    bird_detections = results[results.class_id == 16]
    num_birds = len(bird_detections)
    
    # Collect confidences
    conf_scores = ", ".join([f"{c:.2f}" for c in bird_detections.confidence])
    detection_summary.append({
        "image_name": img_name,
        "bird_count": num_birds,
        "confidence_scores": conf_scores if num_birds > 0 else "N/A"
    })

    # 2. Only Annotate and Save if the image was randomly selected
    if img_path in images_to_save:
        if num_birds > 0:
            labels = [f"{CLASS_NAMES[16]} {conf:.2f}" for conf in bird_detections.confidence]
            annotated_img = box_annotator.annotate(scene=image.copy(), detections=bird_detections)
            annotated_img = label_annotator.annotate(scene=annotated_img, detections=bird_detections, labels=labels)
            cv2.imwrite(os.path.join(OUTPUT_DIR, img_name), annotated_img)
            print(f"Saved (Random Sample): {img_name}")
        else:
            # Optional: Save it anyway to show a "clean" sample, or skip
            cv2.imwrite(os.path.join(OUTPUT_DIR, f"empty_{img_name}"), image)
            print(f"Saved (Random Sample - No Birds): {img_name}")
    
    # Small status update for every 100 images processed
    if len(detection_summary) % 100 == 0:
        print(f"Progress: {len(detection_summary)}/{total_images} images scanned...")

# 5. Export Data (Same as before)
if detection_summary:
    df = pd.DataFrame(detection_summary)
    df.to_csv(os.path.join(OUTPUT_DIR, "detection_summary.csv"), index=False)
    df.to_json(os.path.join(OUTPUT_DIR, "detection_summary.json"), orient="records", indent=4)
    try:
        df.to_excel(os.path.join(OUTPUT_DIR, "detection_summary.xlsx"), index=False)
    except:
        pass

print(f"\nScanning complete! {total_images} results in CSV, and 50 random images saved in: {OUTPUT_DIR}")

[2026-02-14 17:40:35] [WARNING] rf-detr - Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
[2026-02-14 17:40:36] [WARNING] rf-detr - Using patch size 16 instead of 14, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.


[2026-02-14 17:40:36] [INFO] rf-detr - Loading pretrain weights


[2026-02-14 17:40:38] [WARNING] rf-detr - Model is not optimized for inference. Latency may be higher than expected. You can optimize the model for inference by calling model.optimize_for_inference().


Total images to process: 500
Random images to be saved: 50
Verified: ID 16 is 'bird'. Starting inference...
Saved (Random Sample): 003-squacco_heron_000170.jpg
Saved (Random Sample): 003-squacco_heron_000220.jpg
Saved (Random Sample): 003-squacco_heron_000240.jpg
Saved (Random Sample): 003-squacco_heron_000280.jpg
Saved (Random Sample): 003-squacco_heron_000310.jpg
Saved (Random Sample - No Birds): 003-squacco_heron_000710.jpg
Saved (Random Sample - No Birds): 003-squacco_heron_000730.jpg
Saved (Random Sample): 009-yellow_legged_gull_000130.jpg
Progress: 100/500 images scanned...
Saved (Random Sample): 019-eurasian_coot_000030.jpg
Saved (Random Sample): 019-eurasian_coot_000050.jpg
Saved (Random Sample): 019-eurasian_coot_000110.jpg
Saved (Random Sample): 019-eurasian_coot_000130.jpg
Saved (Random Sample): 019-eurasian_coot_000170.jpg
Saved (Random Sample): 019-eurasian_coot_000320.jpg
Saved (Random Sample): 021-little_ringed_plover_000020.jpg
Saved (Random Sample): 021-little_ringed_p